In [1]:
import csv
import math
import itertools
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from subprocess import check_output
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_squared_error

print(check_output(["ls", "/Users/Hermione/MasterUCL/Web economics/Assignment/dataset"]).decode("utf8"))

cleantest.csv
cleantrain.csv
cleanvalidation.csv
test.csv
train.csv
validation.csv



# 1. Feature Selection
Using Gradient Boosting Decision Tree to select important features. 

The input of the GBDT is the one-hot encoded features generated from raw data using DataPreprocessing.py. 

For each input x, it has 584 features, the output of GBDT selected 39 important features.


In [2]:
train = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/train.csv")
validation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/validation.csv")

In [3]:
ytrain = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/train.csv")["click"]
yvalidation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/validation.csv")["click"]

# this correct the missing columns in validation and test data set due to I encoded them seperately
def MissingColumnsCorrector(df1,df2):
    for columns in df1:
        if columns in df2:
            continue
        else:
            missing_columns = columns
            ind = df1.columns.get_loc(missing_columns)
            df2.insert(ind,missing_columns,0.0)
    return df2


xtrain = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/cleantrain.csv")
#xtest = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/cleantest.csv")
xvalidation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/cleanvalidation.csv")

xvalidation = MissingColumnsCorrector(xtrain,xvalidation)

xtrain = np.array(xtrain)
ytrian = np.array(ytrain)
ytrain = [int(numeric_string) for numeric_string in ytrain]
xvalidation = np.array(xvalidation)
yvalidation = np.array(yvalidation)
yvalidation = [int(numeric_string) for numeric_string in yvalidation]

In [ ]:
#using GBDT to find the feature importance
gbdt = GradientBoostingClassifier()
gbdt.fit(xtrain, ytrain)
feature_importance = gbdt.feature_importances_
print(feature_importance)
print("The total feature number is:",gbdt.feature_importances_.shape)


In [ ]:
#get the index of important features, and save as csv for safety 
index = np.where(feature_importance > 0)
index = list(index[0])
print("The number of important feature is:"len(index))


with open('importantFeatureInd', 'w') as myfile:
    wr = csv.writer(myfile,  dialect='excel')
    wr.writerow(index)

In [4]:
index = [0,169,197,309,355,382,400,441,443,447,450,455,458,461,462,466,467,484,486,488,500,501,503,505,506,512,523,530,542,549,555,559,561,564,565,568,569,580,582]
print("The number of important feature is:",len(index))

The number of important feature is: 39


In [5]:
#generate new train data and validation data with only important features
new_train = xtrain[:, index]
new_val = xvalidation[:, index]

# 2. Train logistic regression model
2.1 Train logistic regression model with 584 encoded features and test it on validation set

In [6]:
#train classifier
model = LogisticRegression(class_weight = "balanced")
trainedlr = model.fit(xtrain,ytrain)


In [7]:
prob = trainedlr.predict_proba(xvalidation)

pClick = pd.DataFrame(prob)
predicty = trainedlr.predict(xvalidation)
precision = precision_score(yvalidation, predicty, average='micro')
correctpred = sum(predicty == yvalidation)


In [8]:
print("lr model predicted",correctpred,"correct click status")
print("The precision of lr model is",precision)
print(pClick)

lr model predicted 225313 correct click status
The precision of lr model is 0.751672232434
               0         1
0       0.582652  0.417348
1       0.149288  0.850712
2       0.409382  0.590618
3       0.467619  0.532381
4       0.619432  0.380568
5       0.479779  0.520221
6       0.438161  0.561839
7       0.552823  0.447177
8       0.601464  0.398536
9       0.474166  0.525834
10      0.448029  0.551971
11      0.323780  0.676220
12      0.555458  0.444542
13      0.258690  0.741310
14      0.556464  0.443536
15      0.701694  0.298306
16      0.474980  0.525020
17      0.516823  0.483177
18      0.548258  0.451742
19      0.562471  0.437529
20      0.648080  0.351920
21      0.630845  0.369155
22      0.654046  0.345954
23      0.661568  0.338432
24      0.568472  0.431528
25      0.601418  0.398582
26      0.690712  0.309288
27      0.554625  0.445375
28      0.635074  0.364926
29      0.631475  0.368525
...          ...       ...
299719  0.710449  0.289551
299720  0.535661  

 
2.2 Train logistic regression model with important features selected by GBDT

In [9]:
#train classifier with best parameters
lr = LogisticRegression(C=0.001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
gbdtlr = lr.fit(new_train, ytrain)


In [10]:
gbdt_predict_labels = gbdtlr.predict(new_val)
gbdtprob = gbdtlr.predict_proba(new_val)

gbdtpClick = pd.DataFrame(gbdtprob)

gbdtprecision = precision_score(yvalidation, gbdt_predict_labels, average='micro')
gbdtcorrectpred = sum(gbdt_predict_labels == yvalidation)

In [11]:
print("GBDTlr model predicted",gbdtcorrectpred,"correct click status")
print("lr model predicted",correctpred,"correct click status")


print("The precision of GBDTlr model is",gbdtprecision)
print("The precision of lr model is",precision)
#print(gbdtpClick)

GBDTlr model predicted 257327 correct click status
lr model predicted 225313 correct click status
The precision of GBDTlr model is 0.85847492402
The precision of lr model is 0.751672232434


In [12]:
RMSEgbdt = mean_squared_error(yvalidation, gbdt_predict_labels) 

In [13]:
# apply negative downsampling to work out weights in order for probability of click to have the same ratio as training data, technique is called 
#model recalibration  
temp = len(train) / (2 * np.bincount(train.click))
w = temp[0]/temp[1]
print(w)

GBDTprob =[]
for p in gbdtpClick[1]:
    GBDTprob.append( p / (p + ((1-p)/w)))
GBDTprob[:5]

0.000754533880574


[0.00023558042877098732,
 0.0063420732308209675,
 0.00076951484992390416,
 0.00062236902491641464,
 0.00046413601637275708]

In [ ]:
p = 0.3
pro = p / (p + ((1-p)/w))

In [ ]:
pro

In [14]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve([click for click in validation.click], GBDTprob)
print('AUC accuracy of GBDTlr:',metrics.auc(fpr, tpr))

AUC accuracy of GBDTlr: 0.703199650867


In [15]:
RMSEgbdt = mean_squared_error(yvalidation, GBDTprob) 
RMSEgbdt

0.00075465169451182062

# 3. Optimal Real-Time Bidding(ORTB)
In this section, two ORTB solutions will be used to find the best one.

In [16]:
lambdas = [5.2e-10,5.2e-9,5.2e-8,5.2e-7,5.2e-6,5.2e-5,5.2e-4,5.2e-3,5.2e-2,5.2e-1]
c = np.arange(10,101,10).tolist()
combinations = list(itertools.product(c, lambdas))

In [ ]:
c

In [ ]:
lambdas

In [ ]:
#c=2
#lambdas = 1
#ctr = 2
#ortb1 = math.sqrt(c/lambdas*ctr + c**2)-c
#ortb2 = c*(math.pow((ctr+math.sqrt((c**2)*(lambdas**2)+ctr**2))/(c*lambdas),1/3)-math.pow(c*lambdas/(ctr+math.sqrt((c**2)*(lambdas**2)+ctr**2)),1/3))

In [17]:
GBDTctr = np.asarray(GBDTprob)

In [18]:
def ortb1(c,lambdas):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb1 = []
    
    for pctr in GBDTctr:
        ortb1 = math.sqrt(c/lambdas*pctr + c**2)-c
        bidortb1.append(ortb1)
    
    true_bids = bidortb1 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost
    
    

In [19]:
def ortb2(c,lambdas):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb2 = []
    
    for pctr in GBDTctr:
        ortb2 = c*(math.pow((pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2))/(c*lambdas),1/3)-math.pow(c*lambdas/(pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2)),1/3))
        bidortb2.append(ortb2)
    
    true_bids = bidortb2 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost

In [20]:
def get_results(ortb):
    df = pd.DataFrame()
    
    imp = []
    clik = []
    ct = []
    clamb = []
    for combination in combinations:
        c = combination[0]
        lam = combination[1]
        [imps, clicks, cost] = ortb(c,lam)
        imp.append(imps)
        clik.append(clicks)
        ct.append(cost)
        clamb.append(combination)
    df['C_Lambda'] = clamb
    df['impressions'] = imp
    df['total_cost'] = ct
    df['clicks'] = clik
    df['CTR'] = (df.clicks / df.impressions * 100).round(2).astype(str)
    df['CPM'] = (df.total_cost / df.impressions * 1000).round(2).astype(str)
    df['CPC'] = (df.total_cost / df.clicks).round(2).astype(str)
    return df


In [21]:
ortb1result = get_results(ortb1)
       

In [22]:
ortb2result = get_results(ortb2)

In [23]:
ortb1result.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
45,"(50, 5.2e-05)",5162.0,68001.0,16,0.31,13173.38,4250.06
55,"(60, 5.2e-05)",5297.0,72346.0,16,0.3,13657.92,4521.62
25,"(30, 5.2e-05)",4751.0,54492.0,14,0.29,11469.59,3892.29
65,"(70, 5.2e-05)",5606.0,82027.0,16,0.29,14632.0,5126.69
35,"(40, 5.2e-05)",4996.0,62864.0,14,0.28,12582.87,4490.29
85,"(90, 5.2e-05)",5762.0,86784.0,16,0.28,15061.44,5424.0
75,"(80, 5.2e-05)",5696.0,84454.0,16,0.28,14826.9,5278.38
15,"(20, 5.2e-05)",4443.0,45606.0,12,0.27,10264.69,3800.5
95,"(100, 5.2e-05)",5827.0,88637.0,16,0.27,15211.43,5539.81
5,"(10, 5.2e-05)",3710.0,31745.0,10,0.27,8556.6,3174.5


In [24]:
ortb2result.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
96,"(100, 0.00052)",279.0,4125.0,1,0.36,14784.95,4125.0
95,"(100, 5.2e-05)",9227.0,174311.0,21,0.23,18891.41,8300.52
85,"(90, 5.2e-05)",9125.0,168159.0,21,0.23,18428.38,8007.57
75,"(80, 5.2e-05)",8984.0,160289.0,20,0.22,17841.61,8014.45
65,"(70, 5.2e-05)",8830.0,151811.0,19,0.22,17192.64,7990.05
55,"(60, 5.2e-05)",8670.0,142776.0,18,0.21,16467.82,7932.0
45,"(50, 5.2e-05)",8439.0,129078.0,17,0.2,15295.41,7592.82
35,"(40, 5.2e-05)",8108.0,113754.0,15,0.19,14029.85,7583.6
25,"(30, 5.2e-05)",7647.0,94814.0,13,0.17,12398.85,7293.38
5,"(10, 5.2e-05)",5449.0,39247.0,9,0.17,7202.61,4360.78


In [25]:
ortb1result.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
94,"(100, 5.2e-06)",70132.0,2210365.0,94,0.13,31517.21,23514.52
84,"(90, 5.2e-06)",69268.0,2152080.0,91,0.13,31068.89,23649.23
74,"(80, 5.2e-06)",68218.0,2080404.0,90,0.13,30496.41,23115.6
64,"(70, 5.2e-06)",65478.0,1955148.0,88,0.13,29859.62,22217.59
54,"(60, 5.2e-06)",63653.0,1848000.0,86,0.14,29032.41,21488.37
44,"(50, 5.2e-06)",60795.0,1687483.0,83,0.14,27756.94,20331.12
34,"(40, 5.2e-06)",57563.0,1520874.0,72,0.13,26421.03,21123.25
24,"(30, 5.2e-06)",53360.0,1316378.0,65,0.12,24669.75,20251.97
14,"(20, 5.2e-06)",46755.0,1043466.0,58,0.12,22317.74,17990.79
3,"(10, 5.2e-07)",56828.0,2500018.0,56,0.1,43992.71,44643.18


In [26]:
ortb2result.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
44,"(50, 5.2e-06)",84550.0,2498588.0,88,0.1,29551.6,28393.05
54,"(60, 5.2e-06)",79806.0,2500011.0,82,0.1,31326.1,30487.94
74,"(80, 5.2e-06)",73263.0,2500015.0,82,0.11,34123.84,30487.99
34,"(40, 5.2e-06)",78240.0,2113174.0,80,0.1,27008.87,26414.68
64,"(70, 5.2e-06)",75411.0,2500049.0,80,0.11,33152.31,31250.61
84,"(90, 5.2e-06)",71176.0,2500042.0,79,0.11,35124.79,31646.1
94,"(100, 5.2e-06)",69649.0,2500046.0,76,0.11,35894.93,32895.34
3,"(10, 5.2e-07)",84703.0,2500000.0,66,0.08,29514.89,37878.79
24,"(30, 5.2e-06)",69963.0,1672209.0,65,0.09,23901.33,25726.29
13,"(20, 5.2e-07)",57978.0,2500043.0,54,0.09,43120.55,46297.09


In [27]:
def ortb1opt(c,lambdas):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb1 = []
    
    for pctr in GBDTctr:
        if pctr <= 0.0003:
            ortb1 = 0
        else:
            ortb1 = math.sqrt(c/lambdas*pctr + c**2)-c
        bidortb1.append(ortb1)
    
    true_bids = bidortb1 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost
    
    

In [28]:
ortb1opt = get_results(ortb1opt)

In [36]:
def ortb2opt(c,lambdas):
    impression = 0.0
    clicks = 0
    cost = 0.0
    budget = 2500000  # 1/10th budget of 25,000,000
    bidortb2 = []
    
    for pctr in GBDTctr:
        if pctr <= 0.0003:
            bidprice = 0
        else:
            bidprice = c*(math.pow((pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2))/(c*lambdas),1/3)-math.pow(c*lambdas/(pctr+math.sqrt((c**2)*(lambdas**2)+pctr**2)),1/3))
        bidortb2.append(bidprice)
    
    true_bids = bidortb2 >= validation.payprice
    for i in range(0, len(true_bids)):
        if true_bids[i] == True:
            impression += 1.0
            clicks += validation.click[i]
            cost += validation.payprice[i]
        if cost >= budget:
            break
    return impression, clicks, cost

In [37]:
ortb2opt = get_results(ortb2opt)

In [32]:
ortb1opt.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
94,"(100, 5.2e-06)",54696.0,2025563.0,93,0.17,37033.11,21780.25
84,"(90, 5.2e-06)",53959.0,1969912.0,90,0.17,36507.57,21887.91
74,"(80, 5.2e-06)",53014.0,1900192.0,89,0.17,35843.21,21350.47
64,"(70, 5.2e-06)",51486.0,1798904.0,87,0.17,34939.67,20677.06
54,"(60, 5.2e-06)",50138.0,1700875.0,85,0.17,33923.87,20010.29
44,"(50, 5.2e-06)",47777.0,1550297.0,82,0.17,32448.6,18906.06
34,"(40, 5.2e-06)",45465.0,1399889.0,71,0.16,30790.48,19716.75
24,"(30, 5.2e-06)",42279.0,1213673.0,64,0.15,28706.28,18963.64
14,"(20, 5.2e-06)",36870.0,959678.0,57,0.15,26028.7,16836.46
3,"(10, 5.2e-07)",47867.0,2500002.0,56,0.12,52228.09,44642.89


In [38]:
ortb2opt.sort_values("clicks",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
54,"(60, 5.2e-06)",65003.0,2427039.0,91,0.14,37337.34,26670.76
64,"(70, 5.2e-06)",63225.0,2500017.0,88,0.14,39541.59,28409.28
74,"(80, 5.2e-06)",61556.0,2500086.0,85,0.14,40614.82,29412.78
44,"(50, 5.2e-06)",60952.0,2145908.0,84,0.14,35206.52,25546.52
84,"(90, 5.2e-06)",59890.0,2500012.0,84,0.14,41743.4,29762.05
94,"(100, 5.2e-06)",58569.0,2500004.0,82,0.14,42684.76,30487.85
3,"(10, 5.2e-07)",68492.0,2406133.0,76,0.11,35130.13,31659.64
34,"(40, 5.2e-06)",55446.0,1780870.0,76,0.14,32119.0,23432.5
24,"(30, 5.2e-06)",48664.0,1374966.0,62,0.13,28254.27,22176.87
13,"(20, 5.2e-07)",49730.0,2500072.0,59,0.12,50272.91,42374.1


In [33]:
ortb1opt.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
45,"(50, 5.2e-05)",5044.0,67832.0,16,0.32,13448.06,4239.5
55,"(60, 5.2e-05)",5175.0,72169.0,16,0.31,13945.7,4510.56
25,"(30, 5.2e-05)",4634.0,54325.0,14,0.3,11723.13,3880.36
75,"(80, 5.2e-05)",5572.0,84273.0,16,0.29,15124.37,5267.06
35,"(40, 5.2e-05)",4878.0,62695.0,14,0.29,12852.6,4478.21
65,"(70, 5.2e-05)",5483.0,81848.0,16,0.29,14927.59,5115.5
95,"(100, 5.2e-05)",5701.0,88452.0,16,0.28,15515.17,5528.25
5,"(10, 5.2e-05)",3597.0,31585.0,10,0.28,8780.93,3158.5
85,"(90, 5.2e-05)",5637.0,86601.0,16,0.28,15362.96,5412.56
15,"(20, 5.2e-05)",4327.0,45441.0,12,0.28,10501.73,3786.75


In [39]:
ortb2opt.sort_values("CTR",ascending= False).head(10)

,C_Lambda,impressions,total_cost,clicks,CTR,CPM,CPC
96,"(100, 0.00052)",279.0,4125.0,1,0.36,14784.95,4125.0
85,"(90, 5.2e-05)",8966.0,167905.0,21,0.23,18726.86,7995.48
95,"(100, 5.2e-05)",9068.0,174057.0,21,0.23,19194.64,8288.43
75,"(80, 5.2e-05)",8825.0,160035.0,20,0.23,18134.28,8001.75
65,"(70, 5.2e-05)",8671.0,151557.0,19,0.22,17478.61,7976.68
55,"(60, 5.2e-05)",8511.0,142522.0,18,0.21,16745.62,7917.89
45,"(50, 5.2e-05)",8280.0,128824.0,17,0.21,15558.45,7577.88
35,"(40, 5.2e-05)",7949.0,113500.0,15,0.19,14278.53,7566.67
15,"(20, 5.2e-05)",6580.0,67320.0,11,0.17,10231.0,6120.0
25,"(30, 5.2e-05)",7488.0,94560.0,13,0.17,12628.21,7273.85
